# Notebook for construction of basic dataframe 
## - uses class data structures (see ia_s2p_preprocess.ipynb)

In [3]:
%run ./rob_setup_notebook.ipynb

import utils.utils_funcs as uf
import utils.gsheets_importer as gsi
import utils.ia_funcs as ia 
from utils.paq2py import *

session_type = 'combined'
# 'sensory_nodetrend'
# 'sensory_highactivity'
# 'sensory_topactivity'
# 'sensory_topcells'
# 'sensory_extremefilter'
# 'sensory_responsivecells'
# 'sensory_2sec_test'
# 'projection_nodetrend'
# 'projection_2sec_test'
# 'interneuron'
# 'combined'

combined = True if 'combined' in session_type else False
sensory = True if 'sensory' in session_type else False
projection = True if 'projection' in session_type else False
interneuron = True if 'interneuron' in session_type else False

if interneuron:
    stim_types = ['pr', 'ps', 'none']
if combined:
    stim_types = ['pr', 'ps', 'none']
else:
    stim_types = ['pr', 'ps', 'none', 'w']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Suite2p path: ['/home/rlees/anaconda3/envs/suite2p/lib/python3.7/site-packages/suite2p']
qnap_path: /home/rlees/mnt/qnap 
qnap_data_path /home/rlees/mnt/qnap/Data 
pkl_folder: /home/rlees/mnt/qnap/pkl_files 
master_path: /home/rlees/mnt/qnap/master_pkl/master_obj.pkl 
fig_save_path: /home/rlees/mnt/qnap/Analysis/Figures 
stam_save_path: /home/rlees/mnt/qnap/Analysis/STA_movies 
s2_borders_path: /home/rlees/mnt/qnap/Analysis/S2_borders


# Define dataframe paths

In [4]:
session_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_session_df.pkl'
experiment_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_experiment_df.pkl'
trial_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_trial_df.pkl'
cell_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_cell_df.pkl'
cell_trial_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_cell_trial_df.pkl'
cell_mean_timepoint_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_cell_mean_timepoint_df.pkl'
cell_trial_single_timepoint_df_path = (qnap_path + '/pkl_files/dataframes/' + session_type + 
                                       '_cell_trial_single_timepoint_df.pkl')

# Data organisation

### For the following dataframes it will be possible to uses the dataframe above to index the dataframe below, or vice versa

## Use cases:
- find average response to pr stim in ps targets
    - use the cell dataframe to get cell IDs for ps targets and index the timepoint dataframe to produce an average response to pr
- find sum or mean dff response on a single trial for pr target cells
    - use the cell dataframe to find pr target cells, use timepoint dataframe to group by trial_num and mean the pre and post

# Construct dataframes

## Session dataframe
- all sessions in project (index = session)
- include information such as:
    - ss_id [string]
    - session_id [string] (sheet_name)
    - s2p_path [string]
    - frame_list [string]
    - x_pix_reg [list] (xoff)
    - y_pix_reg [list] (yoff)
    - mean_img [array]
    - mean_imgE [array]

In [8]:
df_name = session_type + '_session_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()
index = []

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    ss_id = ses_obj.ss_id
    session_id = ses_obj.sheet_name
    s2p_path = ses_obj.s2p_path
    frame_list = list(ses_obj.frame_list)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)

    x_pix_reg = list(exp_list[0].xoff)
    y_pix_reg = list(exp_list[0].yoff)
    mean_img = exp_list[0].mean_img[0]
    mean_imgE = exp_list[0].mean_imgE[0]
            
    temp_df = pd.DataFrame({'ss_id'      : ss_id,
                            's2p_path'   : s2p_path,
                            'frame_list' : [frame_list],
                            'x_pix_reg'  : [x_pix_reg],
                            'y_pix_reg'  : [y_pix_reg],
                            'mean_img'   : [mean_img],
                            'mean_imgE'  : [mean_imgE]
                            })
    
    df = pd.concat([df, temp_df])
    index.extend([session_id])
    
df.index = index
df.index.rename('session_id', inplace=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,ss_id,s2p_path,frame_list,x_pix_reg,y_pix_reg,mean_img,mean_imgE
session_id,,,,,,,
2021-02-24_RL132,1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY,/home/rlees/mnt/qnap/Data/2021-02-24/RL132/202...,"[22986, 22986, 22986]","[-2, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -...","[0, 0, 0, 0, -1, -1, 0, 0, 0, -1, -1, -1, -1, ...","[[7.713115229560023, 7.685083674120479, 6.9643...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2021-02-18_RL127,1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY,/home/rlees/mnt/qnap/Data/2021-02-18/RL127/202...,"[22986, 22986, 22986]","[-1, -1, -1, -2, -2, -1, -2, -2, -2, -1, -1, -...","[-1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1...","[[6.5101076017285875, 6.0801357347950935, 4.80...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2020-10-02_RL108,1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY,/home/rlees/mnt/qnap/Data/2020-10-02/RL108/202...,"[22986, 22986, 16977, 22986]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[[9.50269873150106, 9.084062430046012, 9.53177...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2020-09-16_RL103,1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY,/home/rlees/mnt/qnap/Data/2020-09-16/RL103/202...,"[22986, 22986, 16977, 22986]","[0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[4.094172739708992, 4.820032023380177, 4.3196...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2020-10-12_RL109,1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY,/home/rlees/mnt/qnap/Data/2020-10-12/RL109/202...,"[22986, 22986, 16977, 22986]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[8.000799651784604, 6.854781059569703, 6.9906...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## Experiment dataframe
- all experiments i.e. stim types across all sessions (index = session, experiment)
- include information such as:
    - session_id (sheet_name)
    - stim_type
    - tiff_path
    - naparm_path
    - paq_path
    - stim_channel
    - fps
    - frame_x
    - frame_y
    - n_planes
    - pix_sz_x
    - pix_sz_y
    - scan_x
    - scan_y
    - zoom
    - n_frames
    - last_good_frame
    - n_groups
    - n_reps
    - n_shots
    - inter_point_delay
    - single_stim_dur
    - spiral_size
    - stim_dur
    - stim_freq
    - duration_frames
    - frames [tuple]
    - n_units
    - num_s1_cells
    - num_s2_cells
    - pre_frames
    - post_frames
    - test_frames
    - pre_trial_frames
    - post_trial_frames
    - n_trials
    - n_groups = exp_obj.n_groups
    - n_reps = exp_obj.n_reps
    - n_shots = exp_obj.n_shots
    - inter_point_delay = exp_obj.inter_point_delay
    - single_stim_dur = exp_obj.single_stim_dur
    - spiral_size = exp_obj.spiral_size
    - n_targets
    - n_targeted_cells
    - sta_euclid_dist

In [9]:
df_name = session_type + '_experiment_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()
index = []

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    for exp_obj in exp_list:
        
        session_id = exp_obj.sheet_name
        stim_type = exp_obj.stim_type
        tiff_path = str(exp_obj.tiff_path)
        naparm_path = str(exp_obj.naparm_path)
        paq_path = str(exp_obj.paq_path)
        stim_channel = exp_obj.stim_channel
        fps = exp_obj.fps
        frame_x = exp_obj.frame_x
        frame_y = exp_obj.frame_y
        n_planes = exp_obj.n_planes
        pix_sz_x = exp_obj.pix_sz_x
        pix_sz_y = exp_obj.pix_sz_y
        scan_x = exp_obj.scan_x
        scan_y = exp_obj.scan_y
        zoom = exp_obj.zoom
        n_frames = int(exp_obj.n_frames)
        last_good_frame = int(exp_obj.last_good_frame)
        stim_dur = exp_obj.stim_dur
        stim_freq = exp_obj.stim_freq
        duration_frames = exp_obj.duration_frames
        frames = exp_obj.frames
        n_units = exp_obj.n_units[0]
        num_s1_cells = exp_obj.num_s1_cells[0]
        num_s2_cells = exp_obj.num_s2_cells[0]
        pre_frames = exp_obj.pre_frames
        post_frames = exp_obj.post_frames
        test_frames = exp_obj.test_frames
        pre_trial_frames = exp_obj.pre_trial_frames
        post_trial_frames = exp_obj.post_trial_frames
        n_trials = exp_obj.n_trials
        
        try:
            n_groups = exp_obj.n_groups
            n_reps = exp_obj.n_reps
            n_shots = exp_obj.n_shots
            inter_point_delay = exp_obj.inter_point_delay
            single_stim_dur = exp_obj.single_stim_dur
            spiral_size = exp_obj.spiral_size
            n_targets = exp_obj.n_targets
            n_targeted_cells = int(exp_obj.n_targeted_cells)
            sta_euclid_dist = exp_obj.sta_euclid_dist
        except AttributeError:
            print('no photostim attributes for this experiment', session_id, stim_type)
            n_groups = np.nan
            n_reps = np.nan
            n_shots = np.nan
            inter_point_delay = np.nan
            single_stim_dur = np.nan
            spiral_size = np.nan
            n_targets = np.nan
            n_targeted_cells = np.nan
            sta_euclid_dist = np.nan
        
        temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                                'stim_type'  : pd.Series(stim_type, dtype='category'),
                                'tiff_path' : tiff_path,
                                'naparm_path' : naparm_path,
                                'paq_path' : paq_path,
                                'stim_channel' : stim_channel,
                                'fps' : fps,
                                'frame_x' : frame_x,
                                'frame_y' : frame_y,
                                'n_planes' : n_planes,
                                'pix_sz_x' : pix_sz_x,
                                'pix_sz_y' : pix_sz_y,
                                'scan_x' : scan_x,
                                'scan_y' : scan_y,
                                'zoom' : zoom,
                                'n_frames' : n_frames,
                                'last_good_frame' : last_good_frame,
                                'stim_dur' : stim_dur,
                                'stim_freq' : stim_freq,
                                'duration_frames' : duration_frames,
                                'frames' : [frames],
                                'n_units' : n_units,
                                'num_s1_cells' : num_s1_cells,
                                'num_s2_cells' : num_s2_cells,
                                'pre_frames' : pre_frames,
                                'post_frames' : post_frames,
                                'test_frames' : test_frames,
                                'pre_trial_frames' : pre_trial_frames,
                                'post_trial_frames' : post_trial_frames,
                                'n_trials' : n_trials,
                                'sta_euclid_dist' : sta_euclid_dist,
                                'n_groups' : n_groups,
                                'n_reps' : n_reps,
                                'n_shots' : n_shots,
                                'inter_point_delay' : inter_point_delay,
                                'single_stim_dur' : single_stim_dur,
                                'spiral_size' : spiral_size,
                                'n_targets' : n_targets,
                                'n_targeted_cells' : n_targeted_cells
                                })
        
        df = pd.concat([df, temp_df])
        experiment_id = session_id + '_' + stim_type
        index.extend([experiment_id])

df.index = index
df.index.rename('experiment_id', inplace=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,stim_type,tiff_path,naparm_path,paq_path,stim_channel,fps,frame_x,frame_y,n_planes,...,n_trials,sta_euclid_dist,n_groups,n_reps,n_shots,inter_point_delay,single_stim_dur,spiral_size,n_targets,n_targeted_cells
experiment_id,,,,,,,,,,,,,,,,,,,,,
2021-02-24_RL132_pr,2021-02-24_RL132,pr,/home/rlees/mnt/qnap/Data/2021-02-24/RL132/202...,/home/rlees/mnt/qnap/Data/2021-02-24/RL132/202...,/home/rlees/mnt/qnap/Data/2021-02-24/RL132/202...,markpoints2packio,15.023206,1024,1024,1,...,100,101.295644,1,10,1,5,10.0,10,16,34
2021-02-24_RL132_ps,2021-02-24_RL132,ps,/home/rlees/mnt/qnap/Data/2021-02-24/RL132/202...,/home/rlees/mnt/qnap/Data/2021-02-24/RL132/202...,/home/rlees/mnt/qnap/Data/2021-02-24/RL132/202...,markpoints2packio,15.023206,1024,1024,1,...,100,134.223626,1,10,1,5,10.0,10,16,44
2021-02-24_RL132_none,2021-02-24_RL132,none,/home/rlees/mnt/qnap/Data/2021-02-24/RL132/202...,/home/rlees/mnt/qnap/Data/2021-02-24/RL132/202...,/home/rlees/mnt/qnap/Data/2021-02-24/RL132/202...,markpoints2packio,15.023207,1024,1024,1,...,100,69.969445,1,10,1,5,10.0,10,16,44
2021-02-18_RL127_pr,2021-02-18_RL127,pr,/home/rlees/mnt/qnap/Data/2021-02-18/RL127/202...,/home/rlees/mnt/qnap/Data/2021-02-18/RL127/202...,/home/rlees/mnt/qnap/Data/2021-02-18/RL127/202...,markpoints2packio,15.023207,1024,1024,1,...,100,81.219775,1,10,1,5,10.0,10,11,31
2021-02-18_RL127_ps,2021-02-18_RL127,ps,/home/rlees/mnt/qnap/Data/2021-02-18/RL127/202...,/home/rlees/mnt/qnap/Data/2021-02-18/RL127/202...,/home/rlees/mnt/qnap/Data/2021-02-18/RL127/202...,markpoints2packio,15.023207,1024,1024,1,...,100,69.916997,1,10,1,5,10.0,10,11,36


## Trial dataframe
- all trials across all experiments i.e. stim types for all sessions (index = session, experiment, trial)
    - session_id (sheet_name)
    - stim_type
    - trial_num [float]
    - trial_target_dff
    - trial_target_euclid_dist

In [10]:
df_name = session_type + '_trial_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()
index = []

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    for exp_obj in exp_list:
        
        session_id = list(np.repeat(exp_obj.sheet_name, exp_obj.n_trials))
        stim_type = list(np.repeat(exp_obj.stim_type, exp_obj.n_trials))
        experiment_id = [session + '_' + stim for session, stim in zip(session_id,stim_type)]

        trial_num = list(range(1,exp_obj.n_trials+1))
        trial_num_str = np.array(trial_num, dtype='str')
        trial_id = [exp + '_' + trial for exp, trial in zip(experiment_id, trial_num_str)]
        
        try:
            trial_target_dff = exp_obj.trial_target_dff
            trial_target_euclid_dist = exp_obj.trial_euclid_dist
        except AttributeError:
            print('no photostim attributes for this experiment', session_id[0], stim_type[0])
            trial_target_dff = list(np.repeat(np.nan, exp_obj.n_trials))
            trial_target_euclid_dist = list(np.repeat(np.nan, exp_obj.n_trials))
        
        temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                                'stim_type'  : pd.Series(stim_type, dtype='category'),
                                'experiment_id' : experiment_id,
                                'trial_num'  : pd.Series(trial_num, dtype='int8'),
                                'trial_target_dff' : trial_target_dff,
                                'trial_target_euclid_dist' : trial_target_euclid_dist
                                })
        
        df = pd.concat([df, temp_df])
        index.extend(trial_id)
    
df.index = index
df.index.rename('trial_id', inplace=True)
    
# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,stim_type,experiment_id,trial_num,trial_target_dff,trial_target_euclid_dist
trial_id,,,,,,
2021-02-24_RL132_pr_1,2021-02-24_RL132,pr,2021-02-24_RL132_pr,1,6.852334,111.445298
2021-02-24_RL132_pr_2,2021-02-24_RL132,pr,2021-02-24_RL132_pr,2,8.473379,99.754529
2021-02-24_RL132_pr_3,2021-02-24_RL132,pr,2021-02-24_RL132_pr,3,2.423691,91.158673
2021-02-24_RL132_pr_4,2021-02-24_RL132,pr,2021-02-24_RL132_pr,4,7.119164,109.996271
2021-02-24_RL132_pr_5,2021-02-24_RL132,pr,2021-02-24_RL132_pr,5,7.508785,107.531174


## Cell dataframe
- all cells across all sessions (index = session, cell)
- include information such as:
    - session_id (sheet_name)
    - cell_num
    - cell_id [float]
    - s1_cell [bool]
    - s2_cell [bool]
    - pr_target [bool]
    - ps_target [bool]
    - pr_resp_sign [string]
    - ps_resp_sign [string]
    - spont_resp_sign [string]
    - whisker_resp_sign [string]
    - cell_med [tuple]
    - cell_x [list]
    - cell_y [list]
    - prob_response_pr [float]
    - prob_response_ps [float]
    - prob_response_whisker [float]
    - prob_response_spont
    - sta_sig_pr [bool]
    - sta_sig_ps [bool]
    - sta_sig_whisker [bool]
    - sta_sig_spont [bool]
    - sta_sig_nomulti_pr [bool]
    - sta_sig_nomulti_ps [bool]
    - sta_sig_nomulti_whisker [bool]
    - sta_sig_nomulti [bool]
    - wilcoxons

In [11]:
df_name = session_type + '_cell_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()
index = []

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    session_id = list(np.repeat(exp_list[0].sheet_name, exp_list[0].n_units[0]))
    
    cell_num = exp_list[0].cell_id[0]
    cell_num_str = np.array(cell_num, dtype='str')
    cell_id = [session + '_' + cell for session, cell in zip(session_id, cell_num_str)]
    
    s1_cell = exp_list[0].cell_s1[0]
    s2_cell = exp_list[0].cell_s2[0]
    cell_med = exp_list[0].cell_med[0]
    cell_x = exp_list[0].cell_x[0]
    cell_y = exp_list[0].cell_y[0]
    
    for exp_obj in exp_list:
        
        if exp_obj.stim_type == 'pr':
            pr_target = exp_obj.targeted_cells
            pr_resp_pos = exp_obj.sta_amplitudes[0] > 0
            pr_sta_amp_resp = exp_obj.sta_amplitudes[0]
            prob_response_pr = exp_obj.prob_response[0]
            sta_sig_pr = exp_obj.sta_sig[0]
            sta_sig_nomulti_pr = exp_obj.sta_sig_nomulti[0]
            wilcoxons_pr = exp_obj.wilcoxons[0]            
        
        if exp_obj.stim_type == 'ps':
            ps_target = exp_obj.targeted_cells
            ps_resp_pos = exp_obj.sta_amplitudes[0] > 0
            ps_sta_amp_resp = exp_obj.sta_amplitudes[0]
            prob_response_ps = exp_obj.prob_response[0]
            sta_sig_ps = exp_obj.sta_sig[0]
            sta_sig_nomulti_ps = exp_obj.sta_sig_nomulti[0]
            wilcoxons_ps = exp_obj.wilcoxons[0]
        
        if exp_obj.stim_type == 'none':
            spont_resp_pos = exp_obj.sta_amplitudes[0] > 0
            spont_sta_amp_resp = exp_obj.sta_amplitudes[0]
            prob_response_spont = exp_obj.prob_response[0]
            sta_sig_spont = exp_obj.sta_sig[0]
            sta_sig_nomulti_spont = exp_obj.sta_sig_nomulti[0]
            wilcoxons_spont = exp_obj.wilcoxons[0]
        
        if len(exp_list) == 4:
            if exp_obj.stim_type == 'w':
                if not sensory:
                    whisker_sta_amp_resp = exp_obj.sta_amplitudes[0]
                else:
                    whisker_resp_pos = exp_obj.sta_amplitudes[0] > 0
                    whisker_sta_amp_resp = exp_obj.sta_amplitudes[0]
                    prob_response_whisker = exp_obj.prob_response[0]
                    sta_sig_whisker = exp_obj.sta_sig[0]
                    sta_sig_nomulti_whisker = exp_obj.sta_sig_nomulti[0]
                    wilcoxons_whisker = exp_obj.wilcoxons[0]
        else:
            whisker_sta_amp_resp = np.repeat(np.nan, len(exp_obj.sta_amplitudes[0]))
        
    temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                            'cell_num' : pd.Series(cell_num, dtype='int16'),
                            's1_cell' : s1_cell,
                            's2_cell' : s2_cell,
                            'cell_med' : cell_med,
                            'cell_x' : cell_x,
                            'cell_y' : cell_y,
                            'pr_target' : pr_target,
                            'ps_target' : ps_target,
                            'pr_sta_amp_resp' : pr_sta_amp_resp,
                            'ps_sta_amp_resp' : ps_sta_amp_resp,
                            'spont_sta_amp_resp' : spont_sta_amp_resp,
                            'whisker_sta_amp_resp' : whisker_sta_amp_resp,
                            'pr_resp_sign' : pr_resp_pos,
                            'ps_resp_sign' : ps_resp_pos,
                            'spont_resp_sign' : spont_resp_pos,
                            'prob_response_pr' : prob_response_pr,
                            'prob_response_ps' : prob_response_ps,
                            'prob_response_spont' : prob_response_spont,
                            'sta_sig_pr' : sta_sig_pr,
                            'sta_sig_ps' : sta_sig_ps,
                            'sta_sig_spont' : sta_sig_spont,
                            'sta_sig_nomulti_pr' : sta_sig_nomulti_pr,
                            'sta_sig_nomulti_ps' : sta_sig_nomulti_ps,
                            'sta_sig_nomulti_spont' : sta_sig_nomulti_spont,
                            'wilcoxons_pr' : wilcoxons_pr,
                            'wilcoxons_ps' : wilcoxons_ps,
                            'wilcoxons_spont' : wilcoxons_spont
                            })
    
    if sensory:
        temp_df2 = pd.DataFrame({'whisker_resp_sign' : whisker_resp_pos,
                                 'prob_response_whisker' : prob_response_whisker,
                                 'sta_sig_whisker' : sta_sig_whisker,
                                 'sta_sig_nomulti_whisker' : sta_sig_nomulti_whisker,
                                 'wilcoxons_whisker' : wilcoxons_whisker
                                })
    else:
        temp_df2 = pd.DataFrame()
    
    new_df = pd.concat([temp_df, temp_df2], axis=1)
    df = pd.concat([df, new_df])
    index.extend(cell_id)
    
df.index = index
df.index.rename('cell_id', inplace=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,cell_num,s1_cell,s2_cell,cell_med,cell_x,cell_y,pr_target,ps_target,pr_sta_amp_resp,...,prob_response_spont,sta_sig_pr,sta_sig_ps,sta_sig_spont,sta_sig_nomulti_pr,sta_sig_nomulti_ps,sta_sig_nomulti_spont,wilcoxons_pr,wilcoxons_ps,wilcoxons_spont
cell_id,,,,,,,,,,,,,,,,,,,,,
2021-02-24_RL132_1,2021-02-24_RL132,1,True,False,"[332.0, 111.0]","[110, 111, 112, 113, 114, 109, 110, 111, 112, ...","[326, 326, 326, 326, 326, 327, 327, 327, 327, ...",False,False,0.026009,...,0.20,False,False,False,False,False,False,0.945175,0.447332,0.866207
2021-02-24_RL132_2,2021-02-24_RL132,2,True,False,"[592.0, 242.0]","[243, 244, 245, 235, 236, 237, 238, 239, 240, ...","[586, 586, 586, 587, 587, 587, 587, 587, 587, ...",False,False,-0.035719,...,0.51,False,False,False,True,True,False,0.036265,0.007706,0.844622
2021-02-24_RL132_3,2021-02-24_RL132,3,True,False,"[258.0, 300.0]","[298, 299, 300, 301, 302, 303, 304, 297, 298, ...","[253, 253, 253, 253, 253, 253, 253, 254, 254, ...",False,False,0.011239,...,0.67,False,False,False,False,True,False,0.659861,0.004182,0.360405
2021-02-24_RL132_4,2021-02-24_RL132,4,True,False,"[239.0, 429.0]","[434, 428, 429, 430, 431, 432, 433, 434, 435, ...","[233, 234, 234, 234, 234, 234, 234, 234, 234, ...",False,False,-0.005869,...,0.52,False,False,False,True,True,False,0.026108,0.021630,0.300699
2021-02-24_RL132_5,2021-02-24_RL132,5,True,False,"[129.0, 344.0]","[344, 345, 346, 347, 339, 340, 341, 342, 343, ...","[124, 124, 124, 124, 125, 125, 125, 125, 125, ...",False,False,0.055691,...,0.43,False,False,False,False,False,False,0.715513,0.065841,0.483042


## Cell trial dataframe
- all trials across all cells for all sessions (index = session, cell, trial)
- include information such as:
    - session_id (sheet_name)
    - cell_num
    - cell_id
    - trial_num [float]
    - trial_sig_dff [bool]
    - trial_sig_dfsf [bool]
    - amp_resp

In [12]:
df_name = session_type + '_cell_trial_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()
index = []

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    session_id = list(np.repeat(exp_list[0].sheet_name, exp_list[0].n_units[0] * 100))

    cell_num = np.repeat(exp_list[0].cell_id[0], 100, axis=0)
    cell_num_str = np.array(cell_num, dtype='str')
    cell_id = [session + '_' + cell for session, cell in zip(session_id, cell_num_str)]

    trial_num = np.tile(np.arange(1,101), len(exp_list[0].cell_id[0]))
    trial_num_str = np.array(trial_num, dtype='str')

    cell_trial_id = [session + '_' + cell + '_' + trial for session, cell, trial in zip(session_id, cell_num_str, trial_num_str)]
    
    for exp_obj in exp_list:
        
        if exp_obj.stim_type == 'pr':
            trial_sig_dff_pr = pd.Series(exp_obj.trial_sig_dff[0].flatten()).reindex(np.arange(0,len(trial_num)))
            trial_sig_dfsf_pr = pd.Series(exp_obj.trial_sig_dfsf[0].flatten()).reindex(np.arange(0,len(trial_num)))
            pr_amp_resp = pd.Series(exp_obj.all_amplitudes[0].flatten()).reindex(np.arange(0,len(trial_num)))
            
        if exp_obj.stim_type == 'ps':
            trial_sig_dff_ps = pd.Series(exp_obj.trial_sig_dff[0].flatten()).reindex(np.arange(0,len(trial_num)))
            trial_sig_dfsf_ps = pd.Series(exp_obj.trial_sig_dfsf[0].flatten()).reindex(np.arange(0,len(trial_num)))
            ps_amp_resp = pd.Series(exp_obj.all_amplitudes[0].flatten()).reindex(np.arange(0,len(trial_num)))
                    
        if exp_obj.stim_type == 'none':
            trial_sig_dff_spont = pd.Series(exp_obj.trial_sig_dff[0].flatten()).reindex(np.arange(0,len(trial_num)))
            trial_sig_dfsf_spont = pd.Series(exp_obj.trial_sig_dfsf[0].flatten()).reindex(np.arange(0,len(trial_num)))
            spont_amp_resp = pd.Series(exp_obj.all_amplitudes[0].flatten()).reindex(np.arange(0,len(trial_num)))
        
        if sensory:
            if exp_obj.stim_type == 'w':
                trial_sig_dff_whisker = pd.Series(exp_obj.trial_sig_dff[0].flatten()).reindex(np.arange(0,len(trial_num)))
                trial_sig_dfsf_whisker = pd.Series(exp_obj.trial_sig_dfsf[0].flatten()).reindex(np.arange(0,len(trial_num)))
                whisker_amp_resp = pd.Series(exp_obj.all_amplitudes[0].flatten()).reindex(np.arange(0,len(trial_num)))
                    
    temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                            'cell_num' : pd.Series(cell_num, dtype='int16'),
                            'cell_id' : pd.Series(cell_id, dtype='category'),
                            'trial_num' : pd.Series(trial_num, dtype='int8'),
                            'pr_amp_resp' : pr_amp_resp,
                            'ps_amp_resp' : ps_amp_resp,
                            'spont_amp_resp' : spont_amp_resp,
                            'trial_sig_dff_pr' : trial_sig_dff_pr,
                            'trial_sig_dfsf_pr' : trial_sig_dfsf_pr,
                            'trial_sig_dff_ps' : trial_sig_dff_ps,
                            'trial_sig_dfsf_ps' : trial_sig_dfsf_ps,
                            'trial_sig_dff_spont' : trial_sig_dff_spont,
                            'trial_sig_dfsf_spont' : trial_sig_dfsf_spont,
                            })
    
    if sensory:
        temp_df2 = pd.DataFrame({'whisker_amp_resp' : whisker_amp_resp,
                                 'trial_sig_dff_whisker' : trial_sig_dff_whisker,
                                 'trial_sig_dfsf_whisker' : trial_sig_dfsf_whisker
                                })
    else:
        temp_df2 = pd.DataFrame()

    new_df = pd.concat([temp_df, temp_df2], axis=1)
    df = pd.concat([df, new_df])
    index.extend(cell_id)
    
df.index = index
df.index.rename('cell_trial_id', inplace=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,cell_num,cell_id,trial_num,pr_amp_resp,ps_amp_resp,spont_amp_resp,trial_sig_dff_pr,trial_sig_dfsf_pr,trial_sig_dff_ps,trial_sig_dfsf_ps,trial_sig_dff_spont,trial_sig_dfsf_spont
cell_trial_id,,,,,,,,,,,,,
2021-02-24_RL132_1,2021-02-24_RL132,1,2021-02-24_RL132_1,1,-0.114489,-0.138589,-0.000446,False,False,False,False,False,False
2021-02-24_RL132_1,2021-02-24_RL132,1,2021-02-24_RL132_1,2,-0.328837,-0.121924,0.025067,False,False,False,False,False,False
2021-02-24_RL132_1,2021-02-24_RL132,1,2021-02-24_RL132_1,3,0.142693,0.063251,-0.004501,False,False,False,False,False,False
2021-02-24_RL132_1,2021-02-24_RL132,1,2021-02-24_RL132_1,4,-0.303415,-0.049498,0.087689,True,False,False,False,False,False
2021-02-24_RL132_1,2021-02-24_RL132,1,2021-02-24_RL132_1,5,0.133407,-0.054163,-0.051541,False,False,False,False,False,False


## Cell mean timepoint dataframe
- all timepoints across all cells for all trial averages i.e. sessions (index = session, cell, timepoint)
- include information such as:
    - session_id (sheet_name)
    - cell_num
    - cell_id [float]
    - timepoint [timepoints] (a.k.a. time)
    - pr_sta_resp [dff]
    - ps_sta_resp [dff]
    - w_sta_resp [dff]
    - spont_sta_resp [dff]

In [13]:
df_name = session_type + '_cell_mean_timepoint_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    session_id = list(np.repeat(exp_list[0].sheet_name, exp_list[0].n_units[0] * len(exp_list[0].time)))
    
    cell_num = np.repeat(exp_list[0].cell_id[0], len(exp_list[0].time), axis=0)
    cell_id = np.array(cell_num, dtype='str')
    cell_id = np.char.add(np.char.add(session_id, np.repeat('_', len(session_id))), cell_id)
    
    timepoint = np.tile(exp_list[0].time, exp_list[0].n_units[0])
    frame = np.tile(np.arange(len(exp_list[0].time)), exp_list[0].n_units[0])
    
    for exp_obj in exp_list:
        
        if exp_obj.stim_type == 'pr':
            pr_sta_resp = exp_obj.stas[0].flatten()
            
        if exp_obj.stim_type == 'ps':
            ps_sta_resp = exp_obj.stas[0].flatten()
                    
        if exp_obj.stim_type == 'none':
            spont_sta_resp = exp_obj.stas[0].flatten()
                    
        if sensory:
            if exp_obj.stim_type == 'w':
                whisker_sta_resp = exp_obj.stas[0].flatten()

    temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                            'cell_num' : pd.Series(cell_num, dtype='int16'),
                            'cell_id' : pd.Series(cell_id, dtype='category'),
                            'frame' : pd.Series(frame, dtype='int16'),
                            'timepoint' : pd.Series(timepoint, dtype='float32').round(2),
                            'pr_sta_resp' : pr_sta_resp,
                            'ps_sta_resp' : ps_sta_resp,
                            'spont_sta_resp' : spont_sta_resp
                            })

    if sensory:
        temp_df2 = pd.DataFrame({'whisker_sta_resp' : whisker_sta_resp})
    else:
        temp_df2 = pd.DataFrame()
        
    new_df = pd.concat([temp_df, temp_df2], axis=1)
    df = pd.concat([df, new_df])

df.reset_index(drop=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,cell_num,cell_id,frame,timepoint,pr_sta_resp,ps_sta_resp,spont_sta_resp
0,2021-02-24_RL132,1,2021-02-24_RL132_1,0,-2.06,-0.015751,-0.033328,-0.063016
1,2021-02-24_RL132,1,2021-02-24_RL132_1,1,-2.00,0.001540,-0.027285,-0.087300
2,2021-02-24_RL132,1,2021-02-24_RL132_1,2,-1.93,-0.052252,-0.005500,-0.086986
3,2021-02-24_RL132,1,2021-02-24_RL132_1,3,-1.86,-0.019454,-0.021109,-0.071215
4,2021-02-24_RL132,1,2021-02-24_RL132_1,4,-1.80,-0.029477,-0.046276,-0.075016


## Cell trial single timepoint dataframe
- all timepoints across all cells for all trials (index = session, cell, trial, timepoint)
- include information such as:
    - session_id (sheet_name)
    - cell_num
    - cell_id [float]
    - trial_num [float]
    - timepoint [timepoints] (a.k.a. time)
    - pr_resp [dff]
    - ps_resp [dff]
    - w_resp [dff]
    - spont_resp [dff]
- include later?:
    - pr_amp_resp
    - ps_amp_resp
    - w_amp_resp
    - spont_amp_resp

In [5]:
df_name = session_type + '_cell_trial_single_timepoint_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    session_id = list(np.repeat(exp_list[0].sheet_name, exp_list[0].n_units[0] * 100 * len(exp_list[0].time)))
    
    cell_num = np.repeat(exp_list[0].cell_id[0], 100 * len(exp_list[0].time), axis=0)
    cell_id = np.array(cell_num, dtype='str')
    cell_id = np.char.add(np.char.add(session_id, np.repeat('_', len(session_id))), cell_id)
    
    trial_num = np.tile(np.repeat(np.arange(1,101), len(exp_list[0].time)), len(exp_list[0].cell_id[0]))
    
    timepoint = np.tile(exp_list[0].time, exp_list[0].n_units[0] * 100)
    frame = np.tile(np.arange(len(exp_list[0].time)), exp_list[0].n_units[0] * 100)
    
    for exp_obj in exp_list:
        
        all_trials = np.swapaxes(exp_obj.all_trials[0], 1, 2)
        
        if exp_obj.stim_type == 'pr':
            pr_resp = pd.Series(all_trials.flatten()).reindex(np.arange(0,len(trial_num)))
            
        if exp_obj.stim_type == 'ps':
            ps_resp = pd.Series(all_trials.flatten()).reindex(np.arange(0,len(trial_num)))
                    
        if exp_obj.stim_type == 'none':
            spont_resp = pd.Series(all_trials.flatten()).reindex(np.arange(0,len(trial_num)))
        
        if sensory:
            if exp_obj.stim_type == 'w':
                whisker_resp = pd.Series(all_trials.flatten()).reindex(np.arange(0,len(trial_num)))

    temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                            'cell_num' : pd.Series(cell_num, dtype='int16'),
                            'cell_id' : pd.Series(cell_id, dtype='category'),
                            'trial_num' : pd.Series(trial_num, dtype='int8'),
                            'timepoint' : pd.Series(timepoint, dtype='float32').round(2),
                            'pr_resp' : pr_resp,
                            'ps_resp' : ps_resp,
                            'spont_resp' : spont_resp
                            })
    
    if sensory:
        temp_df2 = pd.DataFrame({'whisker_resp' : whisker_resp})
    else:
        temp_df2 = pd.DataFrame()
        
    new_df = pd.concat([temp_df, temp_df2], axis=1)
    df = pd.concat([df, new_df])
    
df.reset_index(drop=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,cell_num,cell_id,trial_num,timepoint,pr_resp,ps_resp,spont_resp
0,2021-02-24_RL132,1,2021-02-24_RL132_1,1,-2.06,0.324005,-0.016197,-0.313046
1,2021-02-24_RL132,1,2021-02-24_RL132_1,1,-2.00,-0.283251,0.176858,-0.011701
2,2021-02-24_RL132,1,2021-02-24_RL132_1,1,-1.93,-0.041561,0.041943,-0.179820
3,2021-02-24_RL132,1,2021-02-24_RL132_1,1,-1.86,0.289139,-0.104400,-0.226929
4,2021-02-24_RL132,1,2021-02-24_RL132_1,1,-1.80,0.097556,0.064824,-0.087404
